In [3]:
from Neuro_Plotting.plot import plot_surf_parc, plot_surf
import numpy as np
import os
import matplotlib.pyplot as plt
from nilearn.surface import load_surf_data
import imageio

In [ ]:
mw_loc = '../raw/fs_LR_32k_label/medialwall.annot'
mw_mask = ~load_surf_data(mw_loc).astype('bool')

def clean_name(parc):
    base = parc.split('/')[-1]
    name = base.replace('_', ' ').replace('.npy', '')
    name = name.replace('-', ' ')
    return name

def plot_prob_parc(parc):
    
    # Only proc. if not already saved
    save_loc = 'Figures/prob_parcels/' + parc.split('/')[-1].replace('.npy', '.gif')
    if os.path.exists(save_loc):
        continue
    
    # Load
    parcel = np.load(parc)

    files = []
    for i in range(parcel.shape[1]):
        
        # Gen title
        title = clean_name(parc) + f' ({i}/{parcel.shape[1]})'
        
        # Plot and save to temp folder
        plot_surf(parcel[:, i],
                  threshold=0.000001,
                  cmap='Reds',
                  title=title)

        save_loc = 'temp/' + str(i) + '.png'
        plt.savefig(save_loc, dpi=50,
                    transparent=True, bbox_inches='tight')
        plt.close()
        
        # Keep track of in files
        files.append(save_loc)
    
    # Load files from temp and make gif
    with imageio.get_writer(save_loc,
                            mode='I',
                            duration=.5) as writer:

        for filename in files:
            image = imageio.imread(filename)
            writer.append_data(image)

# Get all parcels
parcs = os.listdir('../parcels/')
parcs = [os.path.join('../parcels/', p) for p in parcs]

# Make sure directories exist
os.makedirs('Figures/parcels/', exist_ok=True)
os.makedirs('Figures/prob_parcels/', exist_ok=True)
os.makedirs('temp/', exist_ok=True)

for parc in parcs:
    
    # Seperate procedure if prob.
    if '_prob' in parc:
        plot_prob_parc(save_loc)
    
    # Only make and save if doesn't exist
    save_loc = 'Figures/parcels/' + parc.split('/')[-1].replace('.npy', '.png')
    if os.path.exists(save_loc):
        continue

    parcel = np.load(parc)
    parcel[mw_mask] = 0
    
    # Plot and save
    plot_surf_parc(parcel, title=clean_name(parc))
    plt.savefig(save_loc, dpi=100, transparent=True, bbox_inches='tight')
    plt.close()